# 02 Basic Use Cases

## 1. YouTube Link -> Animation

I've always been fascinated by hand drawn animation. Let's see if we can use Open Interpreter to express that.

We'll ask it to redraw every frame in Steve Lacey's "Bad Habit" video with `replicate`. They have a service that can redraw images in new styles.

`Doesn't this require logging into "replicate"?`

Yes, and this is a great reason to use Open Interpreter! We can simply log in to `replicate`, then tell the interpreter how to use it ([I just pasted in replicate's quick start](https://replicate.com/jagilley/controlnet-pose/api)).
Because code is run locally, the interpreter is "logged in" too.

In [ ]:
import interpreter

# Skip confirmation before any code excecution by enabling auto_run
interpreter.auto_run = True

# Chose your model from the list: ['code-llama', 'gpt-3.5-turbo', 'gpt-4']
interpreter.model = "gpt-3.5-turbo"

# Set your temperature. 0.0 is the most conservative, 1.0 is the most creative.
interpreter.temperature = 0

# Paste your OpenAI API key below.
interpreter.api_key = "your_openai_api_key"

> Before you start, make sure to install replicate:

`pip install replicate`

In [ ]:
import os

# Set the API token for Replicate.
# Get yours: https://replicate.com/account
os.environ["REPLICATE_API_TOKEN"] = "api_token" 

Now we'll ask interpreter to redraw the video using Replicate.

In [ ]:
# Preparing the prompt
message = """

I want to edit a video so it's in 12 fps and each frame is redrawn with ControlNet
to give it a stop motion / hand-drawn animation feeling.

Here's how you use ControlNet on a single frame, from Replicate's quick start:

import replicate
output = replicate.run(
    "jagilley/controlnet-canny:aff48af9c68d162388d230a2ab003f68d2638d88307bdaf1c2f1ac95079c9613",
    input={"image": open("path/to/file", "rb"), "prompt": "<prompt>"}
)

(I've noticed that this always returns a list of two links. Just use the second one.)

Please download this video and just use the seconds from 0:10 to 0:17:
https://www.youtube.com/watch?v=VF-FGf_ZZiI

Then reduce it to 12fps, and then replace those frames with ControlNet redraws
using random prompts that evoke stop motion / hand-drawn animation--
think embroidery, pencil art, claymation, yarn on a table, etc--
then add the original sound back.

Thanks!

"""

# Reset the chat
interpreter.reset() 

# Pass in the message above
interpreter.chat(message) 

Just for fun, imagine a function that takes a `youtube_url`, passes it into `interpreter.chat(message)` with the instructions above, then returns the processed filename:

```python
def animate_video(youtube_url):
  interpreter.reset()
  interpreter.message(f'Animate {youtube_url} with the following steps then save to final.mp4. ...')
  return 'final.mp4'
```

## 2. Documents

### 2.1 Create documents

The model has direct access and permissions to the working directory, and you can test it by asking to create some folders with files inside. 

In [ ]:
# Reset the chat
interpreter.reset() 

# Pass a message directly into chat
interpreter.chat("""Can you make a folder called documents and put five .docx files in it
and write a sentence about machine learning in each of them?""") 

### 2.2 Edit all documents in a folder

Now, let's check that model can edit existing files

In [ ]:
# Reset the chat
interpreter.reset()

# Pass a message directly into chat
interpreter.chat("""Go through all the .docx files in my 'documents' folder
and replace every occurrence of 'Machine Learning' with 'AI'.""")

## 3. Slow + Reverb a YouTube Link

In [ ]:
# Put your link here to select a YouTube video
youtube_url = "https://www.youtube.com/watch?v=8GW6sLrK40k"

# Reset the chat
interpreter.reset()

# Pass a message directly into chat
interpreter.chat(f"Can you slow + reverb this song? {youtube_url}")

Now we could open `interpreter.chat()`, ask it to slow it down more, add more reverb, or even create a video with [a .gif backround](https://i.giphy.com/media/zIV7iWK9f0x8Y/giphy.webp).

## 4. Open Interpreter Artwork

The artwork for Open Interpreter was illustrated by Open Interpreter. How?

It was given a description inspired by [Ruby Chen's](https://rubywjchen.com/) GPT-4 artwork:

In [ ]:
message = """

Hello! I'd like your help making some artwork for the Open Interpreter project.

It's going to be pixel art, ~160 wide and 14 pixels tall. Black background.

I'd like to see rectangles on every other row. These should be anywhere from
~6 to 36 pixels in width. They should be placed randomly around the image, never touching eachother
(the space between them should be ~16-64 pixels). They can go off screen / butt up against the edges.

Half of these rectangles should be white, half should be a powerful purple color: R138 G43 B226

Once you've created it, please scale it up with nearest-neighbor 10x.

Please make ~10 options I can review, like banner_1.png, banner_2.png, etc.

"""

interpreter.reset()
interpreter.chat(message)

## 5. Add subtitles to videos

`replicate` also has a speech-to-text service that generates subtitle files (.srt).

Let's ask Open Interpreter to use some code [copied from replicate's quickstart](https://replicate.com/m1guelpf/whisper-subtitles/api) to add hardcoded subtitles to a video:

In [ ]:
message = """
Hello! I just logged into Replicate on this machine. You have my permission to run any code.

Could you use their speech-to-text service to hardcode subtitles to the bottom of billyking.mp4 and make billy_subbed.mp4?
"""

# Again, let's give Open Interpreter an example of how to use the service.
message += """
Here's some code that Replicate provides for how to use their service:

import replicate
output = replicate.run(
    "m1guelpf/whisper-subtitles:7f686e243a96c7f6f0f481bcef24d688a1369ed3983cea348d1f43b879615766",
    input={"audio_path": open("path/to/file", "rb")} # Can also be a video path
)
print(output)

"""

# Now let's reset and run Open Interpreter.
interpreter.reset()
interpreter.chat(message)

You can [watch the output video here.](https://youtube.com/shorts/F6gOzP691po?feature=share)

## 6. YouTube video -> TikTok Clip

In [ ]:
message = """

I'd like your help in making a TikTok clip of this: https://www.youtube.com/watch?v=KgHkAwaW_lk

Please cut the clip from this -- from 0:15 to 0:38 -- and crop it to portrait (exactly 9:16-- this will be tricky)
around the face in the frame. Just follow the face horizontally -- the final video should be as tall as the original.

You'll need to smoothly/softly follow the one face in the frame so please smoothly average out the motion
between confident face detections. Then put the audio back in. Thanks!

"""

interpreter.reset()
interpreter.chat(message)

Nice! It's saved as `cropped_clip_with_audio.mkv`.

One problem -- if we want to send it to our phone and upload it to TikTok or YT Shorts, we'll need it to be an `.mp4` file.

So, let's just ask for that:

In [ ]:
# The interpreter remembers our conversation unless we explicitly .reset() it.
interpreter.chat("Looks great! Can you convert it to an mp4?")

## 7. Bonus: Image -> Animation

Another `replicate` example -- let's use ControlNet to turn [this image](https://i.ibb.co/f0p4Q5R/i-heart-victoria-paris.png) into 90s-style animated intro.

In [ ]:
message = """
Hi, I want to make a video that's comprised of 12 frames. The 12 frames should be of the image
victoria.png run through controlnet with different materials as prompts.

I'm logged into replicate on this machine. Here's how to use replicate's controlnet:

output = replicate.run(
    "jagilley/controlnet-canny:aff48af9c68d162388d230a2ab003f68d2638d88307bdaf1c2f1ac95079c9613",
    input={"image": open("path/to/file", "rb"), "prompt": "metal"}
)
print(output)

Can you run victoria.png through this 12 times with different materials each time like "metal", "embroidery", and "crayon"?
Then stitch together the 12 pictures into a 1 second video clip. Thank you!
"""

interpreter.reset()
interpreter.chat(message)